In [3]:
import numpy as np
with open("scan_001.xyz","r") as fid:
    pointcloud_01 = fid.read()

pointcloud_01 = np.array([[float(p) for p in pc.split()] for pc in pointcloud_01.strip("\n").split("\n")])

with open("scan_002.xyz","r") as fid:
    pointcloud_02 = fid.read()
    
pointcloud_02 = np.array([[float(p) for p in pc.split()] for pc in pointcloud_02.strip("\n").split("\n")])

In [4]:
import torch
print(torch.__version__)

1.13.1+cu116


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [37]:
!pip install --upgrade jinja2
!pip install -q kaolin==0.13.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-1.13.1_cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##F-1 Score
See F-1 score definition for pointcloud at Nvidia's KAOLIN documentation:
https://kaolin.readthedocs.io/en/latest/modules/kaolin.metrics.pointcloud.html
Using 0.1, 0.5, 1.0 for radius

In [7]:
from kaolin.metrics.pointcloud import f_score
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), radius=0.1, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=0.1, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=0.5, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=1.0, eps=1e-08))

tensor([1.0000], device='cuda:0', dtype=torch.float64)
tensor([0.0001], device='cuda:0', dtype=torch.float64)
tensor([0.0177], device='cuda:0', dtype=torch.float64)
tensor([0.0402], device='cuda:0', dtype=torch.float64)


## Chamfer Distance
See Chamfer definition for pointcloud at Nvidia's KAOLIN documentation: 
https://kaolin.readthedocs.io/en/latest/modules/kaolin.metrics.pointcloud.html
We use default parameters for symmetric distance

In [8]:
from kaolin.metrics.pointcloud import chamfer_distance
print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

tensor([0.], device='cuda:0', dtype=torch.float64)
tensor([1121.5381], device='cuda:0', dtype=torch.float64)


## Direct Hausdorff
See Scipy's documentation for direct Hausdorff distance:
https://scipy.github.io/devdocs/reference/generated/scipy.spatial.distance.directed_hausdorff.html

In [9]:
from scipy.spatial.distance import directed_hausdorff
d, _, _ = directed_hausdorff(pointcloud_01, pointcloud_01)
print("Hausdorff distance between PC1 and itself", d)
d, _, _ = directed_hausdorff(pointcloud_01, pointcloud_02)
print("Hausdorff distance between PC1 and PC2", d)

Hausdorff distance between PC1 and itself 0.0
Hausdorff distance between PC1 and PC2 84.35377434908852


## Mean Surface Distance / Average Symmetric Surface Distance
See the definition here:
https://github.com/emrekavur/CHAOS-evaluation/blob/master/CHAOS_Metrics_and_Evaluation_Method.pdf

In [35]:
import numpy as np
from kaolin.metrics.pointcloud import sided_distance

def mean_surface_distance(p1: torch.tensor, p2: torch.tensor) -> float:
  msd = torch.sum(sided_distance(p1, p2)[0]).detach().cpu().numpy() + torch.sum(sided_distance(p2, p1)[0]).detach().cpu().numpy()
  msd /= p1.size()[1] + p2.size()[1]
  return msd

print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

0.0
530.9226978520774


## Residual Mean Square Distance
See definition here: https://github.com/emrekavur/CHAOS-evaluation/blob/master/CHAOS_Metrics_and_Evaluation_Method.pdf

In [36]:
import numpy as np
from kaolin.metrics.pointcloud import sided_distance

def mean_surface_distance(p1: torch.tensor, p2: torch.tensor) -> float:
  msd = torch.sum(torch.square(sided_distance(p1, p2)[0])).detach().cpu().numpy() + torch.sum(torch.square(sided_distance(p2, p1)[0])).detach().cpu().numpy()
  msd /= p1.size()[1] + p2.size()[1]
  return msd

print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

0.0
860766.9340059942
